## Scenario loading and structure 

`GPUDrive` is a multi-agent driving simulator built on top of the [Waymo Open Motion Dataset (WOMD)](https://waymo.com/open/) (See also [Ettinger et al., 2021](https://arxiv.org/abs/2104.10133)). 

In this tutorial, we explain the structure of a traffic scenario and show use processed scenario data with `GPUDrive`.

**Useful links to learn more**:
- [`waymo-open-dataset`](https://github.com/waymo-research/waymo-open-dataset): Official dataset repo
- [tf.Example proto format](https://waymo.com/open/data/motion/tfexample): Data dictionary for a raw WOMD scenario
- [GPUDrive `data_utils`](https://github.com/Emerge-Lab/gpudrive/tree/main/data_utils): Docs and code we use to process the WOMD scenarios

In [1]:
# Dependencies
import json
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import pandas as pd

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)

cmap = ["r", "g", "b", "y", "c"]
%config InlineBackend.figure_format = 'svg'
sns.set("notebook", font_scale=1.1, rc={"figure.figsize": (8, 3)})
sns.set_style("ticks", rc={"figure.facecolor": "none", "axes.facecolor": "none"})

### Iterating through the WOMD dataset

We upload a folder containing three scenarios in the `data/processed/examples` directory that you can work with. The full dataset can be downloaded [here](https://github.com/Emerge-Lab/gpudrive/tree/main?tab=readme-ov-file#dataset). 


Notice that the data folder is structured as follows:

```bash
data/
    - tfrecord-xxxxx-of-xxxxx
    - ....
    - tfrecord-xxxxx-of-xxxxx
```

Every file beginning with `tfrecord` is a unique traffic scenario.

To use the dataset with the simulator, we use the conventions from [PyTorch dataloaders](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). 


Here is example of how to set up a dataloader:

In [3]:
from gpudrive.env.dataset import SceneDataLoader

data_loader = SceneDataLoader(
    root="/workspace/data/gpu_drive/validation", # Path to the dataset
    batch_size=10, # Batch size, you want this to be equal to the number of worlds (envs) so that every world receives a different scene
    dataset_size=4, # Total number of different scenes we want to use
    sample_with_replacement=True, 
    seed=42, 
    shuffle=True,   
)

In [ ]:
# The full dataset that we will be using
data_loader.dataset

In [4]:
# Notice that it only has 4 unique scenes, since we set the dataset_size to 4
set(data_loader.dataset)

{'/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_103.json',
 '/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_108.json',
 '/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_112.json',
 '/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_120.json'}

In [5]:
data_files = next(iter(data_loader))

data_files[0]

'/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_120.json'

In [6]:
from gpudrive.env.env_torch import GPUDriveTorchEnv
from gpudrive.env.config import EnvConfig

In [8]:
# Pass the data_loader to the environment 
env = GPUDriveTorchEnv(
    config=EnvConfig(),
    data_loader=data_loader,
    max_cont_agents=64,  
    device="cuda",
)

Initialization finished


### Deep dive: What is inside a traffic scenario? 🤔🔬

Though every scenario in the WOMD is unique, they all share the same basic data structure. Traffic scenarios are essentially dictionaries, which you can inspect using tools like [JSON Formatter](https://jsonformatter.org/json-viewer). We'll also look at one in this notebook. In a nutshell, traffic scenarios contain a few key elements:

- **Road map**: The layout and structure of the roads.
- **Human driving (expert) demonstrations**: Examples of human driving behavior.
- **Road objects**: Elements such as stop signs and other traffic signals.

In [10]:
# Take an example scene
data_path = "/workspace/data/gpu_drive/validation/tfrecord-00000-of-00150_103.json"

with open(data_path) as file:
    traffic_scene = json.load(file)

traffic_scene.keys()

dict_keys(['name', 'scenario_id', 'objects', 'roads', 'tl_states', 'metadata'])


We will show you how to render a scene in ⏭️ tutorial `03`, which introduces the gym environment wrapper. Let's first take a closer look at the data structure.

### Global Overview

A traffic scene includes the following key elements:

- **`name`**: The name of the traffic scenario.  
- **`scenario_id`**: Unique identifier for every scenario.
- **`objects`**: Dynamic entities such as vehicles or other moving elements in the scene.  
- **`roads`**: Stationary elements, including road points and fixed objects.  
- **`tl_states`**: Traffic light states (currently not included in processing).  
- **`metadata`**: Additional details about the traffic scenario, such as the index of the self-driving car (SDC) and details for the WOSAC Challenge.

In [ ]:
traffic_scene["tl_states"]

In [ ]:
traffic_scene["name"]

In [ ]:
traffic_scene["metadata"]

In [11]:
traffic_scene["scenario_id"]

'7fcb65089a2bb672'

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
pd.Series(
    [
        traffic_scene["objects"][idx]["type"]
        for idx in range(len(traffic_scene["objects"]))
    ]
).value_counts().plot(kind="bar", rot=45, color=cmap)
plt.title(
    f'Distribution of road objects in traffic scene. Total # objects: {len(traffic_scene["objects"])}'
)
plt.show()

This traffic scenario only contains vehicles and pedestrians, some scenes have cyclists as well.

In [ ]:
pd.Series(
    [traffic_scene["roads"][idx]["type"] for idx in range(len(traffic_scene["roads"]))]
).value_counts().plot(kind="bar", rot=45, color=cmap)
plt.title(
    f'Distribution of road points in traffic scene. Total # points: {len(traffic_scene["roads"])}'
)
plt.show()

### In-Depth: Road Objects

This is a list of different road objects in the traffic scene. For each road object, we have information about its position, velocity, size, in which direction it's heading, whether it's a valid object, the type, and the final position of the vehicle.

In [ ]:
# Take the first object
idx = 0

# For each object, we have this information:
traffic_scene["objects"][idx].keys()

In [ ]:
# Position contains the (x, y) coordinates for the vehicle at every time step
print(json.dumps(traffic_scene["objects"][idx]["position"][:10], indent=4))

In [ ]:
# Width and length together make the size of the object, and is used to see if there is a collision
traffic_scene["objects"][idx]["width"], traffic_scene["objects"][idx]["length"]

An object's heading refers to the direction it is pointing or moving in. The default coordinate system in Nocturne is right-handed, where the positive x and y axes point to the right and downwards, respectively. In a right-handed coordinate system, 0 degrees is located on the x-axis and the angle increases counter-clockwise.

Because the scene is created from the viewpoint of an ego driver, there may be instances where the heading of certain vehicles is not available. These cases are represented by the value `-10_000`, to indicate that these steps should be filtered out or are invalid.

In [ ]:
# Heading is the direction in which the vehicle is pointing
plt.plot(traffic_scene["objects"][idx]["heading"])
plt.xlabel("Time step")
plt.ylabel("Heading")
plt.show()

In [ ]:
# Velocity shows the velocity in the x- and y- directions
print(json.dumps(traffic_scene["objects"][idx]["velocity"][:10], indent=4))

In [ ]:
# Valid indicates if the state of the vehicle was observed for each timepoint
plt.xlabel("Time step")
plt.ylabel("IS VALID")
plt.plot(traffic_scene["objects"][idx]["valid"], "_", lw=5)
plt.show()

In [ ]:
# Each object has a goalPosition, an (x, y) position within the scene
traffic_scene["objects"][idx]["goalPosition"]

In [ ]:
# Finally, we have the type of the vehicle
traffic_scene["objects"][idx]["type"]

### In-Depth: Road Points

Road points are static objects in the scene.

In [ ]:
traffic_scene["roads"][idx].keys()

In [ ]:
# This point represents the edge of a road
traffic_scene["roads"][idx]["type"]

In [ ]:
# Geometry contains the (x, y) position(s) for a road point
# Note that this will be a list for road lanes and edges but a single (x, y) tuple for stop signs and alike
print(json.dumps(traffic_scene["roads"][idx]["geometry"][:10], indent=4));